# 0. Информация об видеокарте

In [1]:
!nvidia-smi

Sat Apr 13 20:32:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.12                 Driver Version: 546.12       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080      WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   47C    P8              43W / 370W |   1320MiB / 10240MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 1. Загрузка библиотек и объявление путей

In [2]:
import os
import glob
import random
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import json
from pprint import pprint
import shutil
import yaml
from ultralytics import YOLO

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Пути до датасетов с автомобилями
PATH_CAR = './DATASET/CarDetection' # Путь до датасета Car Detection
PATH_CARS = './DATASET/CarsDetection/Cars Detection/' # Путь до датасета Cars Detection
PATH_CAR_PERSON = './DATASET/CarPerson/Car-Person-v2-Roboflow-Owais-Ahmad/' # Путь до датасета Car Person

# Пути до датасетов с ЛЭП
PATH_STN = './DATASET/STN' # Путь до датасета STN PLAD
PATH_UTILLUTY_POLE = './DATASET/UtilityPole/' # Путь до датасета Utiluty Pole
PATH_TRANSMISSION = './DATASET/Transmission/data_original_size_v1/data_original_size/' # Путь до датасета Transmission

# Пути до датасетов с деревьями
PATH_TREE = './DATASET/Tree/' # Путь до датасета Tree Detection
PATH_TREE_DETECTION = './DATASET/TreeDetection/' # Путь до датасета Tree Detection
PATH_TREE_100 = './DATASET/Trees100/' # Путь до датасета Tree 100

# Пути до датасетов с птицами
PATH_BIRD_ANNOTATION = './DATASET/BirdAnnotation/' # Путь до датасета Bird Annotation 
PATH_BIRDS_DETECTOR = './DATASET/BirdsDetector/' # Путь до датасета Birds Detector
PATH_BIRDS_V3 = './DATASET/BirdsV3/' # Путь до датасета BirdsV3

# Пути до датасетов со зданиями
PATH_BUILDING_DETECTION = './DATASET/BuildingDetection/' # Путь до датасета Building Detection
PATH_JOSH_EDITS = './DATASET/JoshEdits/' # Путь до датасета JoshEdits

# Словарь по понижению размрености датасетов
DATASET_DOWNGRADES = {
    'dataCarDetection': 2, 
    'dataCarPerson': 3, 
    'dataCarsDetection': 4,
    'dataSTN': 1, 
    'dataUnilityPole': 1, 
    'dataTransmission': 1,
    'dataTree': 2, 
    'dataTreeDetection': 2, 
    'dataTree100': 2,
    'dataBirdAnnotation': 4, 
    'dataBirdsDetectod': 4, 
    'dataBirdsV3': 2,
    'dataBuildingDetection': 2, 
    'dataJoshEdits': 2
}

In [4]:
def print_variable_name(variable):
    '''
        Функция позволяет получить имя переменной
        Input: 
            variable - переменная
        Output:
            variable_name - имя переменной
    '''
    variable_name = [name for name, value in globals().items() if value is variable][-1]
    return variable_name

# 2. Обработка данных

In [5]:
def converterYoloToDataFrame(labels_list, path, classes):
    '''
        Функция позволяет перенести все данные с формата YOLO в DataFrame
        Input:
            labels_list - набор названий изображений
            path - путь до папки с изображениями
    '''
    labels = {} # Словарь объктов
    number_item = 0 # Номер записи
    for item in tqdm(labels_list):
        # Читаем все записи
        with open(path + item) as f:
            lines = f.readlines()
    
        # Проходимся по каждой записи
        for line in lines:
            # Получаем координаты в процентах
            x_center, y_center, w, h = (float(number) for number in line.split()[1:]) 
    
            # Заносим в словарь
            labels[number_item] = {
                'image_name': '/'.join(path.split('/')[:-2]) + '/images/' + item[:-3] + 'jpg',
                'x_center': x_center, 
                'y_center': y_center, 
                'w': w, 
                'h': h,
                'classes': classes
            }
            number_item += 1
    return labels

In [6]:
def datasetMinimization(dataSet, count):
    '''
        Функция позволяет понизить размер DataFrame за счёт удаления фотографий
        Input:
            dataSet - первоначальный DataFrame
            count - количество, во сколько раз сжать DataFrame
    '''
    # Список во всеми именами фотографий
    list_image_name = dataSet['image_name'].unique().tolist()

    # Кол-во фотографий в начале
    start_count_image = len(list_image_name)
    
    # Получаем именя изображений, которые оставляем
    name_file = random.sample(list_image_name, len(list_image_name) // count)

    # Записи с этими файлами
    dataSet = dataSet[dataSet['image_name'].isin(name_file)]

    # Кол-во фотографий после обработки
    end_count_image = len(dataSet['image_name'].unique())
    
    # Выведем системноо сообщение об результате сокращения фотографий
    print(f'There were {start_count_image} photos, now there are {end_count_image}')
    
    return dataSet

## 2.0 Классы для модели
    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

## 2.1. Датасеты с автомобилями
### 2.1.1 Обработка датасета Car Detection

In [7]:
img_h, img_w = (380, 676) # Размеры изображения
dataCarDetection = pd.read_csv(PATH_CAR + '/train_solution_bounding_boxes (1).csv') # Читаем датасет

dataCarDetection.rename(columns={'image':'image_name'}, inplace=True) # Переименовываем колонку

dataCarDetection.sample(10) # посмотрим на данные

,image_name,xmin,ymin,xmax,ymax
28,vid_4_11940.jpg,286.150507,192.414736,383.001447,228.605212
173,vid_4_1800.jpg,40.599132,195.838160,130.602026,228.605212
399,vid_4_3260.jpg,0.000000,196.816281,52.827786,235.452059
547,vid_4_9780.jpg,204.818645,193.399450,315.105608,231.793232
384,vid_4_29960.jpg,0.000000,195.838160,45.979740,231.539575
208,vid_4_19740.jpg,617.791606,175.297619,674.043415,215.889640
313,vid_4_26420.jpg,2.445731,197.305341,42.555716,231.050515
191,vid_4_18820.jpg,565.452967,170.896075,676.000000,222.736487
516,vid_4_940.jpg,137.858704,185.523802,308.212673,245.575615
72,vid_4_13660.jpg,292.020261,180.188224,316.966715,207.575611


In [8]:
# Получим координаты центра объекта в процентах 
dataCarDetection['x_center'] = (dataCarDetection['xmin'] + dataCarDetection['xmax']) / 2 / img_w 
dataCarDetection['y_center'] = (dataCarDetection['ymin'] + dataCarDetection['ymax']) / 2 / img_h 

# Получим размеры объекта в процентах
dataCarDetection['w'] = (dataCarDetection['xmax'] - dataCarDetection['xmin']) / img_w # Получаем ширину объекта в процентах
dataCarDetection['h'] = (dataCarDetection['ymax'] - dataCarDetection['ymin']) / img_h # Получаем ширину объекта в процентах
dataCarDetection['classes'] = 0 # Задаём класс объекта
dataCarDetection.drop(
    ['xmin', 'xmax', 'ymin', 'ymax'], # Удаляем ненужные признаки
    axis=1, inplace=True
)

# Изменим путь до файла
dataCarDetection['image_name'] = f'{PATH_CAR}/training_images/' + dataCarDetection['image_name']

# Очистим память 
del img_h, img_w

# Уменьшаем размер DataFrame
dataCarDetection = datasetMinimization(dataCarDetection, DATASET_DOWNGRADES[print_variable_name(dataCarDetection)])

dataCarDetection.sample(10) # Посмотрим на данные

There were 355 photos, now there are 177


,image_name,x_center,y_center,w,h,classes
265,./DATASET/CarDetection/training_images/vid_4_2...,0.884949,0.541104,0.230101,0.211068,0
49,./DATASET/CarDetection/training_images/vid_4_1...,0.534370,0.523729,0.080318,0.073359,0
96,./DATASET/CarDetection/training_images/vid_4_1...,0.385311,0.575853,0.255427,0.180180,0
159,./DATASET/CarDetection/training_images/vid_4_1...,0.798119,0.528234,0.196816,0.144144,0
462,./DATASET/CarDetection/training_images/vid_4_6...,0.350941,0.554617,0.156295,0.099099,0
196,./DATASET/CarDetection/training_images/vid_4_1...,0.562590,0.546252,0.183068,0.144144,0
9,./DATASET/CarDetection/training_images/vid_4_1...,0.219609,0.551400,0.094790,0.102960,0
143,./DATASET/CarDetection/training_images/vid_4_1...,0.410637,0.571992,0.175832,0.115830,0
448,./DATASET/CarDetection/training_images/vid_4_6...,0.141462,0.550756,0.117945,0.109395,0
22,./DATASET/CarDetection/training_images/vid_4_1...,0.562952,0.561052,0.146165,0.109395,0


In [9]:
import inspect
xer = 0

print_variable_name(dataCarDetection)

'dataCarDetection'

## 2.1.2. Обработка датасета Cars Detection

In [10]:
dataCarsDetection = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_CARS}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_CARS}{sample}/labels/', 0)) # Получаем датасет с разметкой
    dataCarsDetection = pd.concat([dataCarsDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataCarsDetection = datasetMinimization(dataCarsDetection, DATASET_DOWNGRADES[print_variable_name(dataCarsDetection)])

dataCarsDetection.sample(10) # Посмотрим на данные

  0%|          | 0/878 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

There were 1254 photos, now there are 313


,image_name,x_center,y_center,w,h,classes
1094,./DATASET/CarsDetection/Cars Detection/train/i...,0.78125,0.591346,0.436298,0.384615,0
103,./DATASET/CarsDetection/Cars Detection/test/im...,0.463942,0.513221,0.300481,0.183894,0
16,./DATASET/CarsDetection/Cars Detection/valid/i...,0.915865,0.391827,0.165865,0.179087,0
443,./DATASET/CarsDetection/Cars Detection/valid/i...,0.884615,0.664663,0.228365,0.317308,0
1204,./DATASET/CarsDetection/Cars Detection/train/i...,0.518029,0.617788,0.149038,0.405048,0
702,./DATASET/CarsDetection/Cars Detection/train/i...,0.504808,0.550481,0.68149,0.477163,0
15,./DATASET/CarsDetection/Cars Detection/test/im...,0.126202,0.313702,0.082933,0.080529,0
1018,./DATASET/CarsDetection/Cars Detection/train/i...,0.550481,0.544471,0.664663,0.629808,0
306,./DATASET/CarsDetection/Cars Detection/train/i...,0.955529,0.527644,0.020433,0.020433,0
523,./DATASET/CarsDetection/Cars Detection/train/i...,0.477163,0.448317,0.897837,0.883413,0


### 2.1.3. Обработка датасета Car Person

In [11]:
dataCarPerson = pd.DataFrame(columns=dataCarDetection.columns)
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_CAR_PERSON}{sample}/labels') # Список файлов с разметкой
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_CAR_PERSON}{sample}/labels/', 0)) # Получаем датасет с разметкой
    dataCarPerson = pd.concat([dataCarPerson, labels.T]) # Соединяем выборки в единый датасет
    
del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataCarPerson = datasetMinimization(dataCarPerson, DATASET_DOWNGRADES[print_variable_name(dataCarPerson)])

dataCarPerson.sample(10) # Посмотрим на данные

  0%|          | 0/1571 [00:00<?, ?it/s]

  0%|          | 0/448 [00:00<?, ?it/s]

  0%|          | 0/224 [00:00<?, ?it/s]

There were 2243 photos, now there are 747


,image_name,x_center,y_center,w,h,classes
3971,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.235577,0.085337,0.086538,0.170673,0
941,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.41226,0.271635,0.05649,0.075721,0
3820,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.252404,0.550481,0.015625,0.070913,0
11455,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.774038,0.274038,0.05649,0.109375,0
3511,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.592548,0.822115,0.100962,0.326923,0
163,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.147837,0.532452,0.121394,0.16226,0
6184,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.564904,0.183894,0.034856,0.019231,0
6197,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.848558,0.882212,0.123798,0.132212,0
764,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.507212,0.668269,0.149038,0.361779,0
6296,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.014423,0.240385,0.028846,0.042067,0


## 2.2. Обработка датасет с ЛЭП
### 2.2.1 Обработка датасета STN_PLAD

In [12]:
# Читаем файл с разметкой датасета STN
with open(PATH_STN + '/annotations.json') as f:
    dataSTN = json.load(f)

pprint(dataSTN['categories']) # Посмотрим на категории

# Получим два DataFrame, один с изображениями, другой с разметкой объектов
data = pd.DataFrame(dataSTN['images']).rename(columns={'id': 'image_id'})
data1 =  pd.DataFrame(dataSTN['annotations'])

# Объединим два дасатсета и удалим лишние признаки
dataSTN = pd.merge(data, data1, on='image_id', how='left').drop(
    [
        'licence', 'segmentation', 'date_captured', 'coco_url', 
        'iscrowd', 'image_id'
    ], axis=1
)

# Очистим память 
del data, data1 

dataSTN.sample(10) # Посмотрим на данные

[{'id': 0, 'name': 'tower', 'supercategory': 'tower'},
 {'id': 1, 'name': 'insulator', 'supercategory': 'component'},
 {'id': 2, 'name': 'spacer', 'supercategory': 'component'},
 {'id': 3, 'name': 'damper', 'supercategory': 'component'},
 {'id': 5, 'name': 'plate', 'supercategory': 'component'}]


,file_name,height,width,flickr_url,area,bbox,category_id,id
814,20181127-A1/DJI_0421.JPG,3078,5472,,2236,"[2613, 1340, 52, 43]",3,818
1633,20181127-A1/DJI_0444.JPG,3078,5472,,2970,"[2620, 1960, 54, 55]",3,1641
1068,20181127-A1/DJI_0453.JPG,3078,5472,,1035,"[1571, 2147, 23, 45]",3,1074
1765,20181127-A1/DJI_0519.JPG,3078,5472,,1035,"[1690, 876, 45, 23]",3,1773
639,20181127-A1/DJI_0498.JPG,3078,5472,,1624,"[4106, 1654, 29, 56]",3,642
1101,20181127-A1/DJI_0441.JPG,3078,5472,,1520,"[2493, 1662, 76, 20]",3,1107
717,20181127-A1/DJI_0493.JPG,3078,5472,,4513589,"[69, 1537, 2929, 1541]",0,720
732,20181127-A1/DJI_0493.JPG,3078,5472,,1740,"[438, 2866, 30, 58]",3,735
1153,20181127-A1/DJI_0467.JPG,3078,5472,,1890,"[2682, 2372, 27, 70]",3,1159
1463,20181127-A1/DJI_0491.JPG,3078,5472,,2208,"[2265, 2261, 69, 32]",3,1470


In [13]:
# Получим координаты объекта
dataSTN[['xmin', 'ymin', 'xmax', 'ymax']] = dataSTN['bbox'].apply(lambda x: pd.Series(x))
dataSTN['xmax'] = dataSTN['xmax'] + dataSTN['xmin']
dataSTN['ymax'] = dataSTN['ymax'] + dataSTN['ymin']

# Получим координаты центра объекта в процентах 
dataSTN['x_center'] = (dataSTN['xmin'] + dataSTN['xmax']) / 2 / dataSTN['width']
dataSTN['y_center'] = (dataSTN['ymin'] + dataSTN['ymax']) / 2 / dataSTN['height']

# Получим размеры объекта в процентах
dataSTN['w'] = (dataSTN['xmax'] - dataSTN['xmin']) / dataSTN['width']
dataSTN['h'] = (dataSTN['ymax'] - dataSTN['ymin']) / dataSTN['height']

# Переименуем признаки
dataSTN.rename(
    columns={
        'file_name': 'image_name',
        'category_id': 'classes'
    }, 
    inplace=True
) 

# Удалим ненужные признаки
dataSTN.drop(
    [
        'height', 'width', 'flickr_url',
        'area', 'bbox', 'id', 'xmin', 
        'xmax', 'ymin', 'ymax'
    ], 
    axis=1, inplace=True
)

# Изменим путь до файла
dataSTN['image_name'] = './DATASET/STN/' + dataSTN['image_name']

# Оставим фотографии принадлижащие только к опоре ЛЭП
dataSTN = dataSTN[dataSTN['classes'] == 0]
dataSTN['classes'] = dataSTN['classes'] + 1

dataSTN.sample(10) # Посомтрим на данные

,image_name,classes,x_center,y_center,w,h
1030,./DATASET/STN/20181127-A1/DJI_0417.JPG,1,0.060855,0.684048,0.121345,0.513645
806,./DATASET/STN/20181127-A1/DJI_0421.JPG,1,0.561038,0.488304,0.388889,0.975958
1031,./DATASET/STN/20181127-A1/DJI_0417.JPG,1,0.700841,0.502437,0.376462,0.995127
1665,./DATASET/STN/20181127-A1/DJI_0448.JPG,1,0.750457,0.728233,0.498721,0.543535
2261,./DATASET/STN/20181127-A1/DJI_0510.JPG,1,0.517818,0.524854,0.089364,0.950292
783,./DATASET/STN/20181127-A1/DJI_0432.JPG,1,0.286915,0.328460,0.095029,0.432749
2262,./DATASET/STN/20181127-A1/DJI_0515.JPG,1,0.706414,0.683398,0.526133,0.633203
692,./DATASET/STN/20181127-A1/DJI_0483.JPG,1,0.497076,0.647498,0.709430,0.705003
1423,./DATASET/STN/20181129-R/DJI_0005.JPG,1,0.905245,0.190378,0.052083,0.104989
969,./DATASET/STN/20181129-R/DJI_0018.JPG,1,0.203490,0.387747,0.015534,0.070998


### 2.2.2. Обработка датасета Utility Pole

In [14]:
dataUnilityPole = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'test']:
    labels_name = os.listdir(f'{PATH_UTILLUTY_POLE}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_UTILLUTY_POLE}{sample}/labels/', 1)) # Получаем датасет с разметкой
    dataUnilityPole = pd.concat([dataUnilityPole, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память
dataUnilityPole.sample(10) # Посмотрим на данные

  0%|          | 0/179 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
200,./DATASET/UtilityPole/train/images/Lampen-_und...,0.646635,0.481971,0.230769,0.838942,1
184,./DATASET/UtilityPole/train/images/istockphoto...,0.36899,0.705529,0.127404,0.396635,1
276,./DATASET/UtilityPole/train/images/wooden-elec...,0.786058,0.569712,0.221154,0.735577,1
84,./DATASET/UtilityPole/train/images/distributio...,0.1875,0.455529,0.149038,0.737981,1
95,./DATASET/UtilityPole/train/images/distributio...,0.496394,0.512019,0.165865,0.682692,1
141,./DATASET/UtilityPole/train/images/images--5-_...,0.354567,0.536058,0.247596,0.8125,1
159,./DATASET/UtilityPole/train/images/images--9-_...,0.088942,0.669471,0.110577,0.348558,1
69,./DATASET/UtilityPole/train/images/Cisco_Rural...,0.153846,0.508413,0.182692,0.963942,1
147,./DATASET/UtilityPole/train/images/images--6-_...,0.484375,0.491587,0.300481,0.896635,1
118,./DATASET/UtilityPole/train/images/images--10-...,0.899038,0.376202,0.197115,0.752404,1


### 2.2.3. Обработка датасета Transmission

In [15]:
list_image = glob.glob(PATH_TRANSMISSION + '*.jpg')
labels = {} # Словарь объктов
number_item = 0 # Номер записи
for path_image in tqdm(list_image):
    with open(path_image[:-3] + 'json') as f:
        json_file = json.load(f)
    xmax = 0
    xmin = 2**10
    ymax = 0
    ymin = 2**10
    height = json_file['imageHeight']
    width = json_file['imageWidth']
    for item in json_file['shapes']:
        if item['label'] in ['tower_wooden', 'cable']:
            for data in item['points']:
                x, y = data
                xmax = x if x > xmax else xmax
                ymax = y if x > ymax else ymax
                xmin = x if y < xmin else xmin
                ymin = y if y < ymin else ymin
            x_center = (xmin + xmax) / 2 / width
            y_center = (ymin + ymax) / 2 / height
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height
            labels[number_item] = {
                'image_name': path_image,
                'x_center': 0.0 if x_center < 0.0 else 1.0 if x_center > 1.0 else x_center, 
                'y_center': 0.0 if y_center < 0.0 else 1.0 if y_center > 1.0 else x_center, 
                'w': 0.0 if w < 0.0 else 1.0 if w > 1.0 else w, 
                'h': 0.0 if h < 0.0 else 1.0 if h > 1.0 else h,
                'classes': 1 if item['label'] == 'tower_wooden' else 5
            }
            number_item += 1
            
dataTransmission = pd.DataFrame(labels).T # Переносим все в DataFrame

# Очищаем память
del list_image, labels, number_item, xmin, xmax, ymin, ymax, h, w, json_file 

# Уменьшаем размер DataFrame
dataTransmission = datasetMinimization(dataTransmission, DATASET_DOWNGRADES[print_variable_name(dataTransmission)])

dataTransmission.sample(10) # Посмотрим на данные

  0%|          | 0/1242 [00:00<?, ?it/s]

There were 1231 photos, now there are 1231


,image_name,x_center,y_center,w,h,classes
9595,./DATASET/Transmission/data_original_size_v1/d...,0.471512,0.471512,0.140961,0.999023,5
8919,./DATASET/Transmission/data_original_size_v1/d...,0.228724,0.228724,0.457449,0.508676,5
992,./DATASET/Transmission/data_original_size_v1/d...,0.632388,0.632388,0.73327,0.269482,5
137,./DATASET/Transmission/data_original_size_v1/d...,0.618444,0.618444,0.040988,0.090473,5
661,./DATASET/Transmission/data_original_size_v1/d...,0.104512,0.104512,0.013136,0.00041,5
290,./DATASET/Transmission/data_original_size_v1/d...,0.480157,0.480157,0.178228,0.783034,5
3851,./DATASET/Transmission/data_original_size_v1/d...,0.49987,0.49987,0.99974,0.069444,5
8617,./DATASET/Transmission/data_original_size_v1/d...,0.140365,0.140365,0.280729,0.279198,5
2576,./DATASET/Transmission/data_original_size_v1/d...,0.663137,0.663137,0.134584,0.851165,5
6938,./DATASET/Transmission/data_original_size_v1/d...,0.308194,0.308194,0.100066,0.77282,5


## 2.3. Обработка датасетов с деревьями
### 2.3.1. Обработка датасета Tree Object Detecion

In [16]:
dataTree = pd.DataFrame(columns=dataCarDetection.columns)
labels_name = os.listdir(f'{PATH_TREE}train/labels/') # Список файлов с разметкой
labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE}train/labels/', 2)) # Получаем датасет с разметкой
dataTree = pd.concat([dataTree, labels.T]) # Соединяем выборки в единый датасет
    
del labels_name, labels # Очищаем память

dataTree.sample(10) # Посмотрим на данные

  0%|          | 0/214 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
100,./DATASET/Tree/train/images/380_jpg.rf.52fd8e0...,0.210156,0.452344,0.370312,0.904687,2
257,./DATASET/Tree/train/images/images-20-_jpg.rf....,0.7875,0.100781,0.31875,0.201563,2
582,./DATASET/Tree/train/images/the-difference-bet...,0.216406,0.624219,0.16875,0.240625,2
467,./DATASET/Tree/train/images/nabil-azmi-WYBikmM...,0.060156,0.7,0.0875,0.307812,2
325,./DATASET/Tree/train/images/images-5-_jpg.rf.4...,0.5875,0.484375,0.382812,0.90625,2
497,./DATASET/Tree/train/images/pexels-egor-kamele...,0.334375,0.527344,0.575,0.5875,2
475,./DATASET/Tree/train/images/oscar-nord-7F39mk3...,0.949219,0.377344,0.084375,0.345313,2
49,./DATASET/Tree/train/images/20230709_155558_jp...,0.550781,0.182031,0.401562,0.364063,2
359,./DATASET/Tree/train/images/images-56-_jpg.rf....,0.499219,0.540625,0.396875,0.804688,2
657,./DATASET/Tree/train/images/trees-isolated-172...,0.635156,0.8,0.60625,0.4,2


### 2.3.2. Обработка датасета TreeDetection 

In [17]:
dataTreeDetection = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_TREE_DETECTION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE_DETECTION}{sample}/labels/', 2)) # Получаем датасет с разметкой
    dataTreeDetection = pd.concat([dataTreeDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataTreeDetection = datasetMinimization(dataTreeDetection, DATASET_DOWNGRADES[print_variable_name(dataTreeDetection)])

dataTreeDetection.sample(10) # Посмотрим на данные

  0%|          | 0/1803 [00:00<?, ?it/s]

  0%|          | 0/174 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

There were 2060 photos, now there are 1030


,image_name,x_center,y_center,w,h,classes
1844,./DATASET/TreeDetection/train/images/2588740f-...,0.907813,0.602344,0.184375,0.710938,2
1209,./DATASET/TreeDetection/train/images/227aee16-...,0.530469,0.521875,0.775,0.95625,2
2994,./DATASET/TreeDetection/train/images/d37f23e4-...,0.739844,0.439844,0.520312,0.879687,2
2232,./DATASET/TreeDetection/train/images/699533e5-...,0.528906,0.525781,0.594531,0.928906,2
2276,./DATASET/TreeDetection/train/images/74022f60-...,0.615625,0.376563,0.628906,0.753125,2
903,./DATASET/TreeDetection/train/images/211024_16...,0.332813,0.534375,0.414844,0.516406,2
215,./DATASET/TreeDetection/train/images/1ec2ec37-...,0.288281,0.160938,0.454688,0.278125,2
2743,./DATASET/TreeDetection/train/images/b7bac2d0-...,0.454688,0.492969,0.909375,0.817187,2
1215,./DATASET/TreeDetection/train/images/228cb9ce-...,0.564844,0.508594,0.815625,0.8375,2
2595,./DATASET/TreeDetection/train/images/a5cc0332-...,0.497656,0.5,0.995313,1.0,2


### 2.3.3. Обработка датасета Tree 100

In [18]:
dataTree100 = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_TREE_100}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE_100}{sample}/labels/', 2)) # Получаем датасет с разметкой
    dataTree100 = pd.concat([dataTree100, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

dataTree100.sample(10) # Посмотрим на данные

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
171,./DATASET/Trees100/train/images/arvore96_jpg.r...,0.4625,0.580469,0.058594,0.06875,2
102,./DATASET/Trees100/train/images/arvore49_jpg.r...,0.067187,0.589063,0.067187,0.170313,2
49,./DATASET/Trees100/train/images/arvore124_jpg....,0.366406,0.582031,0.464844,0.400781,2
26,./DATASET/Trees100/valid/images/arvore57_jpg.r...,0.546094,0.530469,0.535937,0.617188,2
62,./DATASET/Trees100/train/images/arvore25_jpg.r...,0.184375,0.741406,0.0625,0.049219,2
64,./DATASET/Trees100/train/images/arvore28_jpg.r...,0.465625,0.524219,0.6125,0.882031,2
104,./DATASET/Trees100/train/images/arvore49_jpg.r...,0.220312,0.614844,0.066406,0.086719,2
31,./DATASET/Trees100/valid/images/arvore70_jpg.r...,0.647656,0.461719,0.492188,0.584375,2
126,./DATASET/Trees100/train/images/arvore67_jpg.r...,0.285156,0.313281,0.348438,0.426563,2
16,./DATASET/Trees100/train/images/arvore100_jpg....,0.510938,0.44375,0.177344,0.288281,2


## 2.4. Обработка датасета с птицами
### 2.4.1. Обработка датасета с Bird Annotation

In [19]:
dataBirdAnnotation = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRD_ANNOTATION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRD_ANNOTATION}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdAnnotation  = pd.concat([dataBirdAnnotation, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdAnnotation = datasetMinimization(dataBirdAnnotation, DATASET_DOWNGRADES[print_variable_name(dataBirdAnnotation)])

dataBirdAnnotation.sample(10) # Посмотрим на данные

  0%|          | 0/1362 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

There were 1557 photos, now there are 389


,image_name,x_center,y_center,w,h,classes
878,./DATASET/BirdAnnotation/valid/images/voladora...,0.394531,0.213281,0.045312,0.032813,3
16883,./DATASET/BirdAnnotation/train/images/voladora...,0.233594,0.467187,0.045312,0.10625,3
14117,./DATASET/BirdAnnotation/train/images/voladora...,0.45625,0.255469,0.1,0.103125,3
10575,./DATASET/BirdAnnotation/train/images/voladora...,0.571094,0.651563,0.026562,0.032813,3
2627,./DATASET/BirdAnnotation/train/images/e179_jpg...,0.71875,0.596094,0.023438,0.035937,3
16809,./DATASET/BirdAnnotation/train/images/voladora...,0.054688,0.242188,0.082812,0.0875,3
4893,./DATASET/BirdAnnotation/train/images/gg32_jpg...,0.358594,0.815625,0.023438,0.03125,3
8947,./DATASET/BirdAnnotation/train/images/voladora...,0.520312,0.689063,0.059375,0.03125,3
13268,./DATASET/BirdAnnotation/train/images/voladora...,0.323437,0.135156,0.057813,0.051562,3
16815,./DATASET/BirdAnnotation/train/images/voladora...,0.885938,0.228125,0.084375,0.089063,3


### 2.4.2. Обработка датасета с Birds Detection

In [20]:
dataBirdsDetectod = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRDS_DETECTOR}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRDS_DETECTOR}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdsDetectod = pd.concat([dataBirdsDetectod, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdsDetectod = datasetMinimization(dataBirdsDetectod, DATASET_DOWNGRADES[print_variable_name(dataBirdsDetectod)])

dataBirdsDetectod.sample(10)  # Посмотрим на данные

  0%|          | 0/594 [00:00<?, ?it/s]

  0%|          | 0/170 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

There were 849 photos, now there are 212


,image_name,x_center,y_center,w,h,classes
1776,./DATASET/BirdsDetector/valid/images/voladoras...,0.845313,0.841406,0.06875,0.038281,3
1096,./DATASET/BirdsDetector/valid/images/pajarosca...,0.501563,0.707031,0.016406,0.025,3
893,./DATASET/BirdsDetector/valid/images/pajarosca...,0.541406,0.573438,0.021094,0.026562,3
908,./DATASET/BirdsDetector/valid/images/pajarosca...,0.392969,0.360938,0.017969,0.035156,3
2779,./DATASET/BirdsDetector/train/images/voladoras...,0.524219,0.609375,0.052344,0.098437,3
2957,./DATASET/BirdsDetector/train/images/voladoras...,0.434375,0.583594,0.060937,0.092969,3
415,./DATASET/BirdsDetector/train/images/birds-sky...,0.4125,0.783594,0.027344,0.039844,3
952,./DATASET/BirdsDetector/train/images/e137_jpg....,0.435937,0.639062,0.098437,0.125781,3
807,./DATASET/BirdsDetector/valid/images/pajarosca...,0.790625,0.196094,0.075781,0.125,3
2925,./DATASET/BirdsDetector/train/images/voladoras...,0.276562,0.421875,0.069531,0.08125,3


### 2.4.3. Обработка датасета с BirdsV3

In [21]:
dataBirdsV3 = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRDS_V3}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRDS_V3}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdsV3 = pd.concat([dataBirdsV3, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdsV3 = datasetMinimization(dataBirdsV3, DATASET_DOWNGRADES[print_variable_name(dataBirdsV3)])

dataBirdsV3.sample(10)  # Посмотрим на данные

  0%|          | 0/582 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

There were 665 photos, now there are 332


,image_name,x_center,y_center,w,h,classes
1964,./DATASET/BirdsV3/train/images/birds-roof-8390...,0.471875,0.560156,0.03125,0.046875,3
1577,./DATASET/BirdsV3/train/images/Birds-on-a-roof...,0.067969,0.291406,0.051562,0.060937,3
4479,./DATASET/BirdsV3/train/images/starlings-lined...,0.075,0.282813,0.0375,0.051562,3
1808,./DATASET/BirdsV3/train/images/birds-on-roof_j...,0.21875,0.816406,0.165625,0.059375,3
372,./DATASET/BirdsV3/valid/images/Roofing-Issues-...,0.977344,0.235156,0.044531,0.117188,3
3799,./DATASET/BirdsV3/train/images/pigeons-roof-ce...,0.358594,0.810156,0.117188,0.139063,3
4016,./DATASET/BirdsV3/train/images/pigeons-roof-sk...,0.550781,0.394531,0.048438,0.059375,3
1537,./DATASET/BirdsV3/train/images/birds-on-a-red-...,0.167969,0.549219,0.03125,0.023438,3
2504,./DATASET/BirdsV3/train/images/dv___pic___bird...,0.878125,0.5,0.045312,0.04375,3
1429,./DATASET/BirdsV3/train/images/bird-roof-femal...,0.603906,0.502344,0.073438,0.146875,3


## 2.5. Обработка фотографий со зданиями
### 2.5.1. Обработка датасета Building Detection 

In [22]:
dataBuildingDetection = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BUILDING_DETECTION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BUILDING_DETECTION}{sample}/labels/', 4)) # Получаем датасет с разметкой
    dataBuildingDetection = pd.concat([dataBuildingDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBuildingDetection = datasetMinimization(dataBuildingDetection, DATASET_DOWNGRADES[print_variable_name(dataBuildingDetection)])

dataBuildingDetection.sample(10)  # Посмотрим на данные

  0%|          | 0/726 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

There were 802 photos, now there are 401


,image_name,x_center,y_center,w,h,classes
72,./DATASET/BuildingDetection/train/images/120_A...,0.703794,0.449024,0.587604,0.898048,4
783,./DATASET/BuildingDetection/train/images/91_A_...,0.491587,0.498798,0.983173,0.997596,4
265,./DATASET/BuildingDetection/train/images/185_A...,0.498798,0.498798,0.997596,0.997596,4
31,./DATASET/BuildingDetection/valid/images/305_A...,0.5,0.507212,0.992788,0.975962,4
618,./DATASET/BuildingDetection/train/images/384_A...,0.498798,0.5,0.997596,1.0,4
129,./DATASET/BuildingDetection/train/images/13_A_...,0.498798,0.498798,0.997596,0.997596,4
45,./DATASET/BuildingDetection/train/images/111_A...,0.520433,0.503606,0.954327,0.992788,4
464,./DATASET/BuildingDetection/train/images/293_A...,0.480178,0.497326,0.957914,0.994653,4
241,./DATASET/BuildingDetection/train/images/176_A...,0.498798,0.498798,0.997596,0.997596,4
347,./DATASET/BuildingDetection/train/images/20_jp...,0.194801,0.517036,0.386019,0.961121,4


### 2.5.2. Обработка датасета Josh Edits 

In [23]:
dataJoshEdits = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_JOSH_EDITS}{sample}/labels') # Список файлов с разметкой 
        
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_JOSH_EDITS}{sample}/labels/', 4)) # Получаем датасет с разметкой
    dataJoshEdits = pd.concat([dataJoshEdits, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataJoshEdits = datasetMinimization(dataJoshEdits, DATASET_DOWNGRADES[print_variable_name(dataJoshEdits)])

dataJoshEdits.sample(10)  # Посмотрим на данные

  0%|          | 0/3036 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

There were 3251 photos, now there are 1625


,image_name,x_center,y_center,w,h,classes
3524,./DATASET/JoshEdits/train/images/streetview_81...,0.627344,0.338281,0.56875,0.649219,4
40,./DATASET/JoshEdits/train/images/-RN8EYK6_png....,0.335156,0.514844,0.6625,0.578125,4
2794,./DATASET/JoshEdits/train/images/modelok_36-56...,0.185938,0.464062,0.113281,0.042188,4
4615,./DATASET/JoshEdits/train/images/sv_42-1157749...,0.042188,0.444531,0.08125,0.053125,4
5113,./DATASET/JoshEdits/train/images/sv_42-2452928...,0.825781,0.403125,0.328125,0.25625,4
4707,./DATASET/JoshEdits/train/images/sv_42-1236247...,0.413281,0.470313,0.26875,0.215625,4
1420,./DATASET/JoshEdits/train/images/debug_image_2...,0.155469,0.515625,0.310937,0.107031,4
1299,./DATASET/JoshEdits/train/images/debug_image_2...,0.489844,0.35625,0.979688,0.7125,4
161,./DATASET/JoshEdits/test/images/sv_37-35352425...,0.839063,0.534375,0.084375,0.035937,4
645,./DATASET/JoshEdits/train/images/581980606-1_j...,0.114844,0.455469,0.229687,0.480469,4


# 3. Объединение датасетов в единый датасет

In [24]:
data = pd.concat(
    [
        dataCarDetection, dataCarPerson, dataCarsDetection, # Авто
        dataSTN, dataUnilityPole, dataTransmission, # ЛЭП
        dataTree, dataTreeDetection, dataTree100, # Деревья
        dataBirdAnnotation, dataBirdsDetectod, dataBirdsV3, # Птицы
        dataBuildingDetection, dataJoshEdits # Здания
    ], 
    ignore_index=True
) # Объеденим датасеты в один единый

del dataCarDetection, dataCarPerson, dataCarsDetection # очищаем память от датасетов с Авто
del dataSTN, dataUnilityPole, dataTransmission # очищаем память от датасетов с ЛЭП
del dataTree, dataTreeDetection, dataTree100 # очищаем память от датасетов со Деревьями
del dataBirdAnnotation, dataBirdsDetectod, dataBirdsV3 # очищаем память от датасетов со Птицами
del dataBuildingDetection, dataJoshEdits # очищаем память от датасетов со Зданиями

data.sample(10) # Посмотрим на данные

,image_name,x_center,y_center,w,h,classes
1702,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.100962,0.644231,0.034856,0.125,0
11739,./DATASET/Transmission/data_original_size_v1/d...,0.345831,0.345831,0.291318,0.519529,5
9212,./DATASET/Transmission/data_original_size_v1/d...,0.23023,0.23023,0.0,0.523603,5
32717,./DATASET/JoshEdits/train/images/sv_37-4031766...,0.867188,0.374219,0.242969,0.283594,4
4040,./DATASET/CarPerson/Car-Person-v2-Roboflow-Owa...,0.949519,0.061298,0.098558,0.122596,0
19528,./DATASET/TreeDetection/train/images/5ba43b9a-...,0.480469,0.492188,0.960938,0.890625,2
28065,./DATASET/BirdsV3/train/images/Bird-Removal-Ne...,0.802344,0.424219,0.060937,0.05625,3
17088,./DATASET/Transmission/data_original_size_v1/d...,0.772245,0.772245,0.094573,0.13607,5
26689,./DATASET/BirdsDetector/valid/images/pajarosca...,0.571094,0.467187,0.021875,0.038281,3
12473,./DATASET/Transmission/data_original_size_v1/d...,0.393169,0.393169,0.786337,0.965762,5


In [25]:
data['image_name'].unique().shape # Количество фотографий

(7105,)

In [26]:
# Изменим тип данных координат с string на float
for column in data.columns[1:-1]:
    data[column] = data[column].astype(float)

data['classes'] = data['classes'].astype(int) # Изменим тип классов с string на int
data.info() # Посмотрим на сведение дата сета

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34017 entries, 0 to 34016
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_name  34017 non-null  object 
 1   x_center    34017 non-null  float64
 2   y_center    34017 non-null  float64
 3   w           34017 non-null  float64
 4   h           34017 non-null  float64
 5   classes     34017 non-null  int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 1.4+ MB


In [27]:
count_image = data['image_name'].unique().shape[0]
print(f'Dataset contains {count_image} unique photos and {data.shape[0]} objects')
del count_image # Очищаем память

Dataset contains 7105 unique photos and 34017 objects


    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

In [28]:
# Выведем в процентах количество уникальных изображений по классам
data.groupby('image_name')['classes'].head().value_counts().sort_index() /  data.groupby('image_name')['classes'].head().shape[0] * 100

classes
0    21.380309
1     3.909266
2    13.754826
3    15.079365
4    20.179108
5    25.697126
Name: count, dtype: float64

    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

In [29]:
# Распределение классов среди всех объектов
data['classes'].value_counts().sort_index() / data.shape[0] * 100

classes
0    19.878296
1     2.584002
2     8.242937
3    28.523973
4    11.153247
5    29.617544
Name: count, dtype: float64

In [30]:
# Разделим данные на выборки
train_name_file, test_name_file = train_test_split(data['image_name'].unique(), test_size=.30, random_state=42) 
valid_name_file, test_name_file = train_test_split(test_name_file, test_size=.33, random_state=42)

# Посмотрим на процентное отношение выборок 
pd.DataFrame([len(train_name_file), len(valid_name_file), len(test_name_file)], columns=['size']) / len(data['image_name'].unique()) * 100

,size
0,69.992963
1,20.098522
2,9.908515


In [31]:
train_dataCarDetection = data[data['image_name'].isin(train_name_file)] # Записи для тренировочных данных
valid_dataCarDetection = data[data['image_name'].isin(valid_name_file)] # Записи для валидационных данных
test_dataCarDetection = data[data['image_name'].isin(test_name_file)] # Записи для тестовых данных
del data, train_name_file, valid_name_file, test_name_file # Очистим память

In [32]:
import os
list_paths_to_create = [
    'DATA', 'DATA/train', 
    'DATA/valid', 'DATA/test', 'DATA/train/images', 
    'DATA/train/labels', 'DATA/valid/images', 'DATA/valid/labels',
    'DATA/test/images', 'DATA/test/labels'
]

for i in list_paths_to_create:
    os.mkdir(i)


In [33]:
def copyFileDataSet(dataCarDetection, pathEnd):
    '''
        Функция позволяет скопировать фото в новую папку и под каждую из них создать txt файл с разметкой
        Input:
            dataCarDetection - DataFrame содержащий в себе следующиго типа:
                Признаки    Non-Null?  Тип
                image_name  non-null   object 
                x_center    non-null   float64
                y_center    non-null   float64
                w           non-null   float64
                h           non-null   float64
                classes     non-null   int64
            pathEnd - путь созданной папки, внутри которой созданы папки
                images и labels
    '''
    nameFiles = dataCarDetection['image_name'].unique() # Получаем список уникальных файлов

    # Запускаем цикл по все файлам
    for item in tqdm(nameFiles):

        # shutil.copy2(item, pathEnd+'/images') # Копируем данный файл в другую директорию/
        img = cv2.imread(item, cv2.IMREAD_UNCHANGED)
        img = cv2.resize(cv2.UMat(img), (1280, 720), cv2.INTER_NEAREST)
        cv2.imwrite(pathEnd + '/images/' + item.split('/')[-1], img)
        
        dataList = list(dataCarDetection[dataCarDetection['image_name'] == item].values) # Получаем все объекты на фото
        labelsFile = open(pathEnd + '/labels/' + item.split('/')[-1][:-3] + 'txt', 'w+') # Создаём текстовый документ с разметкой
        isFirst = True # Флаг первой строки

        # Запускаем цикл по всем объектам на фото
        for data in dataList:

            # Получаем линию с данными 
            line = '' if isFirst else '\n'
            isFirst = False
            line += str(data[-1]) + ' ' + ' '.join(str(number) for number in data[1:-1]) 
            
            labelsFile.write(line) # Записываем в файл
        labelsFile.close() # Закрываем файл
        
copyFileDataSet(train_dataCarDetection, './DATA/train') # Формируем тренировочную выборку
copyFileDataSet(valid_dataCarDetection, './DATA/valid') # Формируем валидационную выборку
copyFileDataSet(test_dataCarDetection, './DATA/test') # Формируем тестовую выборку
del train_dataCarDetection, valid_dataCarDetection, test_dataCarDetection # Очищаем память

  0%|          | 0/4973 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: './DATA/train/labels/data_original_size\\04_3420.txt'

In [ ]:
# Глобальный путь до директории с проектом 
global_path = os.path.abspath('.') 
yolo_format = dict(
    path=f'{global_path}/DATA',
    train='../train',
    val='../valid',
    test='../test',       
    nc=6,
    names={
        0: 'Car',
        1: 'STN',
        2: 'Tree',
        3: 'Bird',
        4: 'House',
        5: 'Cable',
    }
)

del global_path # Очищаем память
             
with open('./DATA/yolo.yaml', 'w') as outfile:
    yaml.dump(yolo_format, outfile, default_flow_style=False)

In [ ]:
epochTraning = 250 # Количество эпох обучения
pathYalm = "./DATA/yolo.yaml" # Путь до файла yalm
patience = 10 # Терпимость эпох для переобучения
imgsz = 1280 # Размер изображения

In [ ]:
model = YOLO('yolov8n.yaml').load('yolov8n.pt')
model.train(
    data=pathYalm, 
    epochs=epochTraning, 
    patience=patience, 
    imgsz=imgsz
)  